# Implementación Práctica y Exploración de RAG con Langchain


## 1. Implementacion Práctica de RAG

En esta seccion se implementa desde cero un sistema RAG usando Langchain


In [ ]:
# Configuración base y utilidades
# pip install langchain langchain_community langchain-text-splitters langchain-ollama langchain-chroma langchain-google-genai chromadb pandas

import os
from pathlib import Path

PROJECT_DIR = Path().resolve()
DATA_FILE = PROJECT_DIR / "El coleccionista de sonidos.txt"
DEFAULT_SPLIT = {"chunk_size": 320, "chunk_overlap": 40}

API_KEY = os.getenv("API_KEY") or os.getenv("GOOGLE_API_KEY")
if not API_KEY:
    print("Aviso: define API_KEY o GOOGLE_API_KEY para usar Gemini; también puedes probar solo con Ollama.")


### Carga y preparación de un documento de ejemplo

Para efectos de prueba, creamos un archivo textual con datos random.


In [ ]:
relato_base = """
Clara nació en un faro que ya no guía barcos, pero sí recoge historias. Su pasatiempo favorito consiste en grabar murmullos del pueblo costero y clasificarlos como si fueran minerales.
Conserva en frascos el zumbido de las redes mojadas, el crujido del pan cuando sale del horno y los latidos apurados de quienes anuncian tormenta.
Un jueves de niebla quiso atrapar el silencio que antecede a una confesión y descubrió que no cabe en ningún recipiente; apenas roza el vidrio y se transforma en luz.
Desde ese día comenzó a coleccionar silencios en un cuaderno: anota dónde aparecieron, qué olor tenían y qué conversación salvaron.
No dejó de escuchar sonidos, pero aprendió que algunos momentos se entienden mejor cuando se mezclan ambos.
"""

with open(DATA_FILE, "w", encoding="utf-8") as archivo_destino:
    archivo_destino.write(relato_base.strip())

print(f"Archivo de ejemplo actualizado en {DATA_FILE}")


### Pipeline RAG: Carga, Split, Embedding, Indexado y QA


In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI

def preparar_fragmentos(path=DATA_FILE, split_config=DEFAULT_SPLIT):
    loader = TextLoader(str(path))
    documentos = loader.load()
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=split_config["chunk_size"],
        chunk_overlap=split_config["chunk_overlap"],
    )
    return documentos, splitter.split_documents(documentos)

def inicializar_llm(api_key):
    try:
        if not api_key:
            raise ValueError("API_KEY no configurada")
        return ChatGoogleGenerativeAI(model="gemini-2.0-flash", api_key=api_key)
    except Exception as exc:
        print(f"No se pudo iniciar Gemini: {exc}. Usa ChatOllama o cualquier otro LLM disponible.")
        return None

EMBED_MODEL_NAME = "nomic-embed-text"
docs, chunks = preparar_fragmentos()
embedding_model = OllamaEmbeddings(model=EMBED_MODEL_NAME)
db = Chroma.from_documents(chunks, embedding_model)
llm = inicializar_llm(API_KEY)


**Función de pregunta-respuesta sobre el contenido usando RAG**


In [ ]:
def consultar_relato(pregunta, store=db, modelo=llm, k=3):
    if store is None:
        raise ValueError("La base vectorial no fue inicializada")
    docs = store.similarity_search(pregunta, k=k)
    contexto = "\n---\n".join(d.page_content for d in docs)
    prompt = (
        "Responde solo con el siguiente contexto.\n"
        f"Contexto:\n{contexto}\n\nPregunta: {pregunta}\nRespuesta:"
    )
    if modelo is None:
        raise RuntimeError("No hay LLM activo. Configura Gemini u Ollama antes de consultar.")
    respuesta = modelo.invoke(prompt)
    return respuesta.content if hasattr(respuesta, "content") else respuesta

pregunta_ejemplo = "¿Qué empezó a coleccionar Clara además de sonidos?"
print(consultar_relato(pregunta_ejemplo))


---
## 2. uso de Modelos de Embedding y LLM

Se prueban distintos modelos de embeddings y LLM compatibles para observar como recupera documentos y como genera las respuestas.


In [ ]:
from langchain_community.embeddings import SentenceTransformerEmbeddings

embedding_setups = [
    ("nomic-embed-text", OllamaEmbeddings(model="nomic-embed-text")),
    ("embeddinggemma", OllamaEmbeddings(model="embeddinggemma")),
]

query_text = "memorias guardadas en frascos"

for name, emb in embedding_setups:
    try:
        db_temp = Chroma.from_documents(chunks, emb, persist_directory=f"./chroma_{name}")
        docs_found = db_temp.similarity_search(query_text, k=2)
        print(f"\nEmbedding: {name}")
        for d in docs_found:
            print(f"> Fragmento: {d.page_content[:90]}...")
    except Exception as exc:
        print(f"Error con embedding {name}: {exc}")


In [ ]:
import time
import pandas as pd

def benchmark_embeddings(consulta, configuraciones):
    resultados = []
    for nombre, emb in configuraciones:
        try:
            start = time.time()
            db_temp = Chroma.from_documents(chunks, emb, persist_directory=f"./benchmark_{nombre}")
            db_temp.similarity_search(consulta, k=2)
            elapsed = round(time.time() - start, 3)
            resultados.append({"Modelo": nombre, "Tiempo (seg)": elapsed})
            print(f"Embedding {nombre} listo en {elapsed} seg")
        except Exception as exc:
            resultados.append({"Modelo": nombre, "Tiempo (seg)": "Error", "Detalle": str(exc)})
            print(f"Fallo {nombre}: {exc}")
    return pd.DataFrame(resultados)

benchmark_embeddings("inteligencia artificial experta en relatos", embedding_setups)


In [ ]:
    from langchain_ollama import ChatOllama

    llm_disponibles = [
        ("Gemini 2.0 Flash", llm),
        ("qwen2.5:0.5b", ChatOllama(model="qwen2.5:0.5b")),
    ]

    for nombre, modelo_llm in llm_disponibles:
        if modelo_llm is None:
            print(f"Modelo {nombre} no inicializado. Se omite.")
            continue
        try:
            print(f"
>>> {nombre}")
            respuesta = consultar_relato("¿De qué trata el relato completo?", store=db, modelo=modelo_llm)
            print(respuesta)
        except Exception as exc:
            print(f"Error usando {nombre}: {exc}")


---
## 3. Optimizacion del Separador de Texto

Se prueba variación en los parametros `chunk_size` y `chunk_overlap` del CharacterTextSplitter.


In [ ]:
    sizes = [
        (400, 80),
        (700, 120),
    ]

    for cs, co in sizes:
        splitter = CharacterTextSplitter(chunk_size=cs, chunk_overlap=co)
        pedazos = splitter.split_documents(docs)
        print(f"chunk_size={cs}, chunk_overlap={co} => chunks: {len(pedazos)}")
        print(f"> Ejemplo de chunk: {pedazos[0].page_content[:60]}...
")


---
## 4. Gestionar Bases de Datos Vectoriales

Esta seccion es para la persistencia, colecciones y búsquedas avanzadas con Chroma.


In [ ]:
persist_path = "./chroma_db"
db_persist = Chroma.from_documents(chunks, embedding_model, persist_directory=persist_path)

print(f"Base de datos vectorial guardada en {persist_path}")


In [ ]:
for d in chunks:
    d.metadata["origen"] = DATA_FILE.name

db_filtrada = Chroma.from_documents(chunks, embedding_model)
results = db_filtrada.similarity_search_with_score(
    "frascos",
    k=5,
    filter={"origen": DATA_FILE.name},
)
for doc, score in results:
    print(f"Score: {score:.3f} | Fragmento: {doc.page_content[:70]}...")


---
## 5. Refinamiento de Prompts

Experimentamos con diferentes plantillas y técnicas de prompt engineering para mejorar la calidad, relevancia y trazabilidad de las respuestas generadas por el LLM dentro del sistema RAG.


In [ ]:
    from langchain_core.prompts import PromptTemplate

    base_prompt = """Eres un analista literario y solo puedes usar el bloque {context} para responder.
    Señala la evidencia con comillas si es útil. Debes decir "No lo sé" si el texto no contiene la respuesta.

    Texto:
    {context}

    Pregunta:
    {question}

    Conclusión fundamentada:"""
    plantilla = PromptTemplate(template=base_prompt, input_variables=["context", "question"])

    def consultar_con_plantilla(pregunta, store=db, modelo=llm, k=4, prompt_template=plantilla):
        docs = store.similarity_search(pregunta, k=k)
        contexto = "

".join(d.page_content for d in docs)
        prompt = prompt_template.format(context=contexto, question=pregunta)
        respuesta = modelo.invoke(prompt)
        return respuesta.content if hasattr(respuesta, 'content') else respuesta

    pregunta_2 = "¿Quién es Clara?"
    print(consultar_con_plantilla(pregunta_2))
